This notebook is intended to train and save a basic keras model, for later TRT conversion

In [1]:
import sys
sys.path.append('../../..')

## Data

In [2]:
from src.data.cifar100 import CLASSES, INPUT_SHAPE, load_data

(train_images, train_labels), (test_images, test_labels) = load_data()

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

## Params

In [3]:
_NAME = 'resnet50'
_EPOCH = 1

In [4]:
from datetime import datetime
_TIME = datetime.now().strftime("%Y-%m-%dT%H:%M")

## Create

In [5]:
from src.models.basic import construct_basic_model, NAME as BASIC
from src.models.batchn import construct_batchn_model, NAME as BATCHN
from src.models.conv import construct_conv_model, NAME as CONV
from src.models.resnet50 import construct_resnet50_model, NAME as RESNET50

if _NAME == BASIC:
    construct_model = construct_basic_model
elif _NAME == BATCHN:
    construct_model = construct_batchn_model
elif _NAME == CONV:
    construct_model = construct_conv_model
elif _NAME == RESNET50:
    construct_model = construct_resnet50_model
else:
    raise ValueError('Unexpected: %s' % _NAME)

keras_model = construct_model(INPUT_SHAPE, len(CLASSES))
keras_model.summary()
keras_model

W0903 19:31:52.961632 139713803220800 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Model: "resnet50"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 2048)              23587712  
_________________________________________________________________
fc100 (Dense)                (None, 100)               204900    
Total params: 23,792,612
Trainable params: 23,739,492
Non-trainable params: 53,120
_________________________________________________________________


## Use custom wrapper KerasModel

This way we will save a metadata file with the input/output names

In [6]:
from src.meta.metadata import Metadata
from src.keras.model import KerasModel

metadata = Metadata(_NAME, _EPOCH)

keras_model_wrapper = KerasModel(metadata, keras_model)

## Compile

In [7]:
keras_model_wrapper.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

0

## Train

In [ ]:
keras_model_wrapper.fit(train_images, train_labels, epochs=_EPOCH)

39584/50000 [======================>.......] - ETA: 15s - loss: 5.0334 - acc: 0.0274

## Save

In [ ]:
%%bash

TRAIN_DIR=../../../tmp/trt_end_to_end/train
mkdir -p ${TRAIN_DIR}
#find ${TRAIN_DIR} -type f | xargs rm
find ${TRAIN_DIR} -type f

In [ ]:
from src.utils.trt_end_to_end_constants import *
from datetime import datetime


filename = H5_FILE_FORMAT % (_NAME, _EPOCH, _TIME)
filepath = os.path.join(get_train_dir(_NAME, _EPOCH), filename)

save_params(_NAME, _EPOCH, _TIME)
print('Saving to', filepath)
keras_model_wrapper.save(filepath)

In [ ]:
%%bash

TRAIN_DIR=../../../tmp/trt_end_to_end/train
find ${TRAIN_DIR} -type f && echo
echo "Params:"
find ${TRAIN_DIR} -type f | sort | grep params | xargs cat && echo && echo
echo "Metadata:"
find ${TRAIN_DIR} -type f | sort | grep md | xargs cat && echo

#find ${TRAIN_DIR} -type f | sort | grep md | xargs jq '.name'
#find ${TRAIN_DIR} -type f | sort | grep md | xargs jq '.epoch'